# Homework 4

In [ ]:
!pip install pyperplan


## Let's Make a Plan


### Question
Use `run_planning` to find a plan for the blocks problem defined at the top of the colab file (`BLOCKS_DOMAIN`, `BLOCKS_PROBLEM`).

  The `run_planning` function takes in a PDDL domain string, a PDDL problem string, the name of a search algorithm, and the name of a heuristic (if the search algorithm is informed) or a customized heuristic class. It then uses the Python planning library `pyperplan` to find a plan.

  The plan returned by `run_planning` is a list of pyperplan Operators. You should not need to manipulate these data structures directly in this homework, but if you are curious about the definition, see [here](https://github.com/aibasel/pyperplan/blob/master/pyperplan/task.py#L23).

  The search algs available in pyperplan are: `astar, wastar, gbf, bfs, ehs, ids, sat`. The heuristics available in pyperplan are: `blind, hadd, hmax, hsa, hff, lmcut, landmark`.

  For this question, use the `astar` search algorithm with the `hadd` heuristic.

For reference, our solution is **2** line(s) of code.

In [ ]:
def planning_warmup():
    """Use run_planning to find a plan for the blocks problem defined at the
    top of the colab file (BLOCKS_DOMAIN, BLOCKS_PROBLEM).

      Use the astar search algorithm with the hadd heuristic.

    Returns:
      plan: A list of actions; each action is a pyperplan Operator.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def warmup_test1():
    plan = planning_warmup()
    assert len(plan) == 8
    assert plan[0].name == '(unstack a b)'

warmup_test1()

print('Tests passed.')

## Fill in the Blanks


### Question
You've received PDDL domain and problem strings from your boss and you need to make a plan, pronto! Unfortunately, some of the PDDL is missing.

  Here's what you know. What you're trying to model is a newspaper delivery robot. The robot starts out at a "home base" where there are papers that it can pick up. The robot can hold arbitrarily many papers at once. It can then move around to different locations and deliver papers.

  Not all locations want a paper -- the goal is to satisfy all the locations that do want a paper.

  You also know:
  * There are 6 locations in addition to 1 for the homebase. Locations 1, 2, 3, and 4 want paper; locations 5 and 6 do not.
  * There are 8 papers at the homebase.
  * The robot is initially at the homebase with no papers packed.

  Use this description to complete the PDDL domain and problem.

  If you are running into issues writing or debugging the PDDL you can check out this online PDDL editor, which comes with a built-in planner: [editor.planning.domains](http://editor.planning.domains). To use the editor, you can create two files, one for the domain and one for the problem. You can then click "Solve" at the top to use the built-in planner.

For a general reference on PDDL, check out [planning.wiki](https://planning.wiki/). Note that the PDDL features supported by pyperplan are very limited: types and constants are supported, but that's about it. If you want to make a domain that involves more advanced features, you can try the built-in planner at [editor.planning.domains](http://editor.planning.domains), or you can use any other PDDL planner of your choosing.

Debugging PDDL can be painful. The online editor at [editor.planning.domains](http://editor.planning.domains) is helpful: pay attention to the syntax highlighting and to the line numbers in error messages that result from trying to Solve. To debug, you can also comment out multiple lines of your files by highlighting and using command+/. Common issues to watch out for include:
* A predicate is not defined in the domain file
* A variable name does not start with a question mark
* An illegal character is used (we recommended sticking with alphanumeric characters, dashes, or underscores; and don't start any names with numbers)
* An operator is missing a parameter (in general, the parameters should be exactly the variables that are used anywhere in the operator's preconditions or effects)
* An operator is missing a necessary precondition or effect
* Using negated preconditions, which are not allowed in basic Strips

If you get stuck debugging your PDDL file for more than 10-15 min, please reach out and we'll help!

  

For reference, our solution is **89** line(s) of code.

In [ ]:
def pddl_warmup():
    """Creates a PDDL domain and problem strs for newspaper delivery (uses
    TYPING).

    Returns:
      domain: str
      problem: str
    """
    domain_str = """(define (domain newspapers)
    (:requirements :strips :typing)
    (:types loc paper)
    (:predicates 
      (isHomeBase ?loc - loc)
      ; TODO: Add missing predicates!
    )

    (:action pick-up
      :parameters ()    ; TODO: Add missing parameters!
      :precondition (and
        (at ?loc)
        (isHomeBase ?loc)
        (unpacked ?paper)
      )
      :effect (and
        (not (unpacked ?paper))
        (carrying ?paper)
      )
    )

    (:action move
      :parameters (?from - loc ?to - loc)
      :precondition (and
        (at ?from) 
      )
      :effect (and
        (not (at ?from))
        (at ?to)
      )
    )

    (:action deliver
      :parameters (?paper - paper ?loc - loc)
      :precondition (and
        ; TODO: Add missing preconditions!
      )
      :effect (and
        ; TODO: Add missing effects!
      )
    )

)"""

    problem_str = """(define (problem newspapers1) (:domain newspapers)
  (:objects
    loc-0 - loc
    loc-1 - loc
    loc-2 - loc
    loc-3 - loc
    loc-4 - loc
    loc-5 - loc
    loc-6 - loc
    paper-0 - paper
    paper-1 - paper
    paper-2 - paper
    paper-3 - paper
    paper-4 - paper
    paper-5 - paper
    paper-6 - paper
    paper-7 - paper
  )
  (:init 
    (at loc-0)
    (unpacked paper-0)
    ; TODO: Add missing initial atoms!
  )
  (:goal (and
    (satisfied loc-1)
    (satisfied loc-2)
    (satisfied loc-3)
    (satisfied loc-4)
  ))
)"""

    return domain_str, problem_str

### Tests

In [ ]:
def warmup_test2():
    domain, problem = pddl_warmup()
    plan = run_planning(domain, problem, "gbf", "hadd")
    assert plan, "Failed to find a plan."
    picked_up_papers = set()
    satisfied_locs = set()
    for op in plan:
        if "pickup" in op.name:
            _, _, paper, _ = op.name.split(" ")
            assert paper not in picked_up_papers, \
                "Should not pick up the same paper twice"
            picked_up_papers.add(paper)
        elif "deliver" in op.name:
            _, loc = op.name.rsplit(" ", 1)
            assert loc.endswith(")")
            loc = loc[:-1]
            assert loc not in satisfied_locs, \
                "Should not deliver to the same place twice"
            satisfied_locs.add(loc)
    assert satisfied_locs == {"loc-1", "loc-2", "loc-3", "loc-4"}

warmup_test2()

print('Tests passed.')

## Heuristics Comparision


### Utilities

### Pyperplan Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
import os
import tempfile
import math
import random

import numpy as np
from pyperplan.pddl.parser import Parser
from pyperplan import grounding, planner

# This uses TYPING
BLOCKS_DOMAIN = """(define (domain blocks)
    (:requirements :strips :typing)
    (:types block)
    (:predicates 
        (on ?x - block ?y - block)
        (ontable ?x - block)
        (clear ?x - block)
        (handempty)
        (holding ?x - block)
    )

    (:action pick-up
        :parameters (?x - block)
        :precondition (and
            (clear ?x) 
            (ontable ?x) 
            (handempty)
        )
        :effect (and
            (not (ontable ?x))
            (not (clear ?x))
            (not (handempty))
            (holding ?x)
        )
    )

    (:action put-down
        :parameters (?x - block)
        :precondition (and 
            (holding ?x)
        )
        :effect (and 
            (not (holding ?x))
            (clear ?x)
            (handempty)
            (ontable ?x))
        )

    (:action stack
        :parameters (?x - block ?y - block)
        :precondition (and
            (holding ?x) 
            (clear ?y)
        )
        :effect (and 
            (not (holding ?x))
            (not (clear ?y))
            (clear ?x)
            (handempty)
            (on ?x ?y)
        )
    )

    (:action unstack
        :parameters (?x - block ?y - block)
        :precondition (and
            (on ?x ?y)
            (clear ?x)
            (handempty)
        )
        :effect (and 
            (holding ?x)
            (clear ?y)
            (not (clear ?x))
            (not (handempty))
            (not (on ?x ?y))
        )
    )
)
"""

# This uses TYPING
BLOCKS_PROBLEM = """(define (problem blocks)
    (:domain blocks)
    (:objects 
        d - block
        b - block
        a - block
        c - block
    )
    (:init 
        (clear a) 
        (on a b) 
        (on b c)
        (on c d)
        (ontable d) 
        (handempty)
    )
    (:goal (and (on d c) (on c b) (on b a)))
)
"""

# The BW domain does not use TYPING
BW_BLOCKS_DOMAIN = """(define (domain prodigy-bw)
  (:requirements :strips)
  (:predicates (on ?x ?y)
               (ontable ?x)
               (clear ?x)
               (handempty)
               (holding ?x)
               )
  (:action pick-up
             :parameters (?ob1)
             :precondition (and (clear ?ob1) (ontable ?ob1) (handempty))
             :effect
             (and (not (ontable ?ob1))
                   (not (clear ?ob1))
                   (not (handempty))
                   (holding ?ob1)))
  (:action put-down
             :parameters (?ob)
             :precondition (holding ?ob)
             :effect
             (and (not (holding ?ob))
                   (clear ?ob)
                   (handempty)
                   (ontable ?ob)))
  (:action stack
             :parameters (?sob ?sunderob)
             :precondition (and (holding ?sob) (clear ?sunderob))
             :effect
             (and (not (holding ?sob))
                   (not (clear ?sunderob))
                   (clear ?sob)
                   (handempty)
                   (on ?sob ?sunderob)))
  (:action unstack
             :parameters (?sob ?sunderob)
             :precondition (and (on ?sob ?sunderob) (clear ?sob) (handempty))
             :effect
             (and (holding ?sob)
                   (clear ?sunderob)
                   (not (clear ?sob))
                   (not (handempty))
                   (not (on ?sob ?sunderob)))))
"""

BW_BLOCKS_PROBLEM_1 = """(define (problem bw-simple)
  (:domain prodigy-bw)
  (:objects A B C)
  (:init (clear a) (handempty) (on a b) (ontable b))
  (:goal (and (ontable a) (clear b))))
"""

BW_BLOCKS_PROBLEM_2 = """(define (problem bw-sussman)
  (:domain prodigy-bw)
  (:objects A B C)
  (:init (ontable a) (ontable b) (on c a)
                (clear b) (clear c) (handempty))
  (:goal (and (on a b) (on b c))))
"""

BW_BLOCKS_PROBLEM_3 = """(define (problem bw-large-a)
  (:domain prodigy-bw)
  (:objects 1 2 3 4 5 6 7 8 9)
  (:init (handempty)
         (on 3 2)
         (on 2 1)
         (ontable 1)
         (on 5 4)
         (ontable 4)
         (on 9 8)
         (on 8 7)
         (on 7 6)
         (ontable 6)
         (clear 3)
         (clear 5)
         (clear 9))
  (:goal (and
          (on 1 5)
          (ontable 5)
          (on 8 9)
          (on 9 4)
          (ontable 4)
          (on 2 3)
          (on 3 7)
          (on 7 6)
          (ontable 6)
          (clear 1)
          (clear 8)
          (clear 2)
          )))
"""

BW_BLOCKS_PROBLEM_4 = """(define (problem bw-large-b)
  (:domain prodigy-bw)
  (:objects 1 2 3 4 5 6 7 8 9 10 11)
  (:init (handempty)
         (on 3 2)
         (on 2 1)
         (ontable 1)
         (on 11 10)
         (on 10 5)
         (on 5 4)
         (ontable 4)
         (on 9 8)
         (on 8 7)
         (on 7 6)
         (ontable 6)
         (clear 3)
         (clear 11)
         (clear 9))
  (:goal (and
          (on 1 5)
          (on 5 10)
          (ontable 10)
          (on 8 9)
          (on 9 4)
          (ontable 4)
          (on 2 3)
          (on 3 11)
          (on 11 7)
          (on 7 6)
          (ontable 6)
          (clear 1)
          (clear 8)
          (clear 2)
          )))
"""

BW_BLOCKS_PROBLEM_5 = """(define (problem bw-large-c)
  (:domain prodigy-bw)
  (:objects 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15)
  (:init (handempty)
	 (on 3 2)
	 (on 2 1)
	 (on 1 12)
	 (on 12 13)
	 (ontable 13)
	 (on 11 10)
	 (on 10 5)
	 (on 5 4)
	 (on 4 14)
	 (on 14 15)
	 (ontable 15)
	 (on 9 8)
	 (on 8 7)
	 (on 7 6)
	 (ontable 6)
	 (clear 3)
	 (clear 11)
	 (clear 9))
  (:goal (and
	  (on 14 1)
	  (on 1 5)
	  (on 5 10)
	  (ontable 10)
	  (on 15 13)
	  (on 13 8)
	  (on 8 9)
	  (on 9 4)
	  (ontable 4)
	  (on 12 2)
	  (on 2 3)
	  (on 3 11)
	  (on 11 7)
	  (on 7 6)
	  (ontable 6)
	  (clear 14)
	  (clear 15)
	  (clear 12)
	  )))

"""

BW_BLOCKS_PROBLEM_6 = """(define (problem bw-large-d)
  (:domain prodigy-bw)
  (:objects 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19)
  (:init (handempty)
	 (on 1 12)
	 (on 12 13)
	 (ontable 13)
	 (on 11 10)
	 (on 10 5)
	 (on 5 4)
	 (on 4 14)
	 (on 14 15)
	 (ontable 15)
	 (on 9 8)
	 (on 8 7)
	 (on 7 6)
	 (ontable 6)
	 (on 19 18)
	 (on 18 17)
	 (on 17 16)
	 (on 16 3)
	 (on 3 2)
	 (ontable 2)
	 (clear 1)
	 (clear 11)
	 (clear 9)
	 (clear 19))
  (:goal (and
	  (on 17 18)
	  (on 18 19)
	  (on 19 14)
	  (on 14 1)
	  (on 1 5)
	  (on 5 10)
	  (ontable 10)
	  (on 15 13)
	  (on 13 8)
	  (on 8 9)
	  (on 9 4)
	  (ontable 4)
	  (on 12 2)
	  (on 2 3)
	  (on 3 16)
	  (on 16 11)
	  (on 11 7)
	  (on 7 6)
	  (ontable 6)
	  (clear 17)
	  (clear 15)
	  (clear 12)
	  )))
"""


def run_planning(domain_pddl_str,
                 problem_pddl_str,
                 search_alg_name,
                 heuristic_name=None):
    """Plan a sequence of actions to solve the given PDDL problem.

    This function is a lightweight wrapper around pyperplan.

    Args:
      domain_pddl_str: A str, the contents of a domain.pddl file.
      problem_pddl_str: A str, the contents of a problem.pddl file.
      search_alg_name: A str, the name of a search algorithm in
        pyperplan. Options: astar, wastar, gbf, bfs, ehs, ids, sat.
      heuristic_name: A str, the name of a heuristic in pyperplan.
        Options: blind, hadd, hmax, hsa, hff, lmcut, landmark.

    Returns:
      plan: A list of actions; each action is a pyperplan Operator.
    """
    # Parsing the PDDDL
    domain_file = tempfile.NamedTemporaryFile(delete=False)
    problem_file = tempfile.NamedTemporaryFile(delete=False)
    with open(domain_file.name, 'w') as f:
        f.write(domain_pddl_str)
    with open(problem_file.name, 'w') as f:
        f.write(problem_pddl_str)
    parser = Parser(domain_file.name, problem_file.name)
    domain = parser.parse_domain()
    problem = parser.parse_problem(domain)
    os.remove(domain_file.name)
    os.remove(problem_file.name)

    # Ground the PDDL
    task = grounding.ground(problem)

    # Get the search alg
    search_alg = planner.SEARCHES[search_alg_name]

    if heuristic_name is None:
        return search_alg(task)

    # Get the heuristic
    heuristic = planner.HEURISTICS[heuristic_name](task)

    # Run planning
    return search_alg(task, heuristic)


def test_run_planning(domain_pddl_str, problem_pddl_str, search_alg_name,
                      heuristic_name):
    import sys, logging, time
    logging.basicConfig(level=logging.INFO,
                        handlers=[logging.StreamHandler(sys.stdout)])
    start_time = time.time()
    plan = run_planning(domain_pddl_str, problem_pddl_str, search_alg_name,
                        heuristic_name)
    run_time = time.time() - start_time
    print(f'Run time\t {run_time:.4f}\t Plan length\t {len(plan)}')
    return run_time, len(plan)


def get_task_definition_str(domain_pddl_str, problem_pddl_str):
    """Get Pyperplan task definition from PDDL domain and problem.

    This function is a lightweight wrapper around Pyperplan.

    Args:
      domain_pddl_str: A str, the contents of a domain.pddl file.
      problem_pddl_str: A str, the contents of a problem.pddl file.

    Returns:
      task: a structure defining the problem
    """
    # Parsing the PDDDL
    domain_file = tempfile.NamedTemporaryFile(delete=False)
    problem_file = tempfile.NamedTemporaryFile(delete=False)
    with open(domain_file.name, 'w') as f:
        f.write(domain_pddl_str)
    with open(problem_file.name, 'w') as f:
        f.write(problem_pddl_str)
    parser = Parser(domain_file.name, problem_file.name)
    domain = parser.parse_domain()
    problem = parser.parse_problem(domain)
    os.remove(domain_file.name)
    os.remove(problem_file.name)

    # Ground the PDDL
    task = grounding.ground(problem)

    return task


def get_task_definition(domain_pddl, problem_pddl):
    """Get Pyperplan task definition from PDDL domain and problem.

    This function is a lightweight wrapper around Pyperplan.

    Args:
      domain_pddl_str: A str, the name of a domain.pddl file.
      problem_pddl_str: A str, the name of a problem.pddl file.

    Returns:
      task: a structure defining the problem
    """
    # Parsing the PDDDL
    parser = Parser(domain_pddl, problem_pddl)
    domain = parser.parse_domain()
    problem = parser.parse_problem(domain)

    # Ground the PDDL
    task = grounding.ground(problem)

    return task


def h_add_test(prob_str, expected_h):
    task = get_task_definition_str(BW_BLOCKS_DOMAIN, prob_str)
    h = h_add(task)
    assert h == expected_h, f"Expected h={expected_h}, but got {h}."
    return h


def h_ff_test(prob_str, expected_h):
    task = get_task_definition_str(BW_BLOCKS_DOMAIN, prob_str)
    h = h_ff(task)
    assert h == expected_h, f"Expected h={expected_h}, but got {h}."

### Question

    We have created some helper functions to plot the running time and plan length of certain search algorithms and heuristics. 
    You can uncomment the last line and set the time limit in the following code block to get the planning results.
    

In [ ]:
import logging, sys


def test_combinations(test_searchalgs, test_heuristics, timeout):
    import signal

    class TimeoutException(Exception):
        pass

    def timeout_handler(signum, frame):
        raise TimeoutException

    # NOTE: only works on Unix-based systems.
    signal.signal(signal.SIGALRM, timeout_handler)
    result = {}
    for i, problem in enumerate([
            BW_BLOCKS_PROBLEM_1, BW_BLOCKS_PROBLEM_2, BW_BLOCKS_PROBLEM_3,
            BW_BLOCKS_PROBLEM_4, BW_BLOCKS_PROBLEM_5, BW_BLOCKS_PROBLEM_6
    ]):
        for alg in test_searchalgs:
            for h in test_heuristics:
                print(f'problem_{i + 1}', alg, h)
                signal.alarm(timeout)
                logging.basicConfig(
                    level=logging.INFO,
                    handlers=[logging.StreamHandler(sys.stdout)])
                try:
                    run_time, plan_length = test_run_planning(
                        BW_BLOCKS_DOMAIN, problem, alg, h)
                except TimeoutException:
                    print(f'Terminate after {timeout} sec.')
                    result[(alg, h, i + 1)] = [-1, -1]
                    print()
                    continue
                else:
                    signal.alarm(0)
                result[(alg, h, i + 1)] = [run_time, plan_length]
                print()
    return result


def plot_result(result, timeout):
    import matplotlib.pyplot as plt
    all_problem_keys = result.keys()
    for problem_id in range(1, 7):
        problem_keys = [k for k in all_problem_keys if k[2] == problem_id]
        problem_keys_str = list(map(lambda x: f'{x[0]}-{x[1]}', problem_keys))
        bb = [
            result[k][0] if result[k][0] != -1 else timeout
            for k in problem_keys
        ]
        cc = [result[k][1] if result[k][0] != -1 else 0 for k in problem_keys]
        fig, axes = plt.subplots(figsize=(10, 6), ncols=2, sharey=True)
        axes[0].barh(problem_keys_str, bb, align='center', color='y')
        axes[1].barh(problem_keys_str, cc, align='center', color='g')
        axes[0].invert_xaxis()
        plt.subplots_adjust(wspace=0)

        axes[0].set(title='Planning time')
        axes[1].set(title='Plan length')
        fig.suptitle(f'Problem-{problem_id}')
        plt.show()


def test_all_combinations(timeout=5):
    all_searchalgs = ['gbf', 'astar']
    all_heuristics = ['blind', 'hmax', 'hadd', 'hff', 'lmcut']
    result = test_combinations(all_searchalgs, all_heuristics, timeout)
    plot_result(result, timeout)


# Uncomment the following line to plot results
# test_all_combinations(timeout=30)

## Construct Reduced Planning Graph (RPG)


### Question
Given a [Pyperplan Task](https://github.com/aibasel/pyperplan/blob/master/pyperplan/task.py) instance and an optional state, return (F_t, A_t), two lists as defined in the RPG pseudo-code from lecture.  Note that _fact names_ in the documentation of `Task` are strings such as '(on a b)' and operator instances are instances of the `Operator` class; the `name` of an operator instance is a string such as '(unstack a c)'.  Is state is None, use `task.initial_state`.

For reference, our solution is **15** line(s) of code.

In [ ]:
def construct_rpg(task, state=None):
    """Constructs a Relaxed Planning Graph (RPG) for a given Pyperplan Task and
    state.  If state is None, use the initial_state of the task.

    Args:
      task: a Pyperplan Task instance
      state: a set of facts (can be None)

    Returns:
      rpg: A pair (F_sets, A_sets).  Note that F_sets has length one longger than A_sets.  F_sets is a list and the item in the list at F[t] should be a set of fact names.  We expect F_set to be something like "[{'(clear a)', ...}, {'(ontable b)',...}, ...]”.  A_sets is a list and the item in the list at A[t] should be a set of operator instance names.  We expect A_sets to be something like "[{'(unstack a b)', ...}, {'(stack a b)', ... }, ... ]"
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def construct_rpg_test1():
    task = get_task_definition_str(BW_BLOCKS_DOMAIN, BW_BLOCKS_PROBLEM_1)
    expected_F_sets = [{'(clear a)', '(on a b)', '(ontable b)', '(handempty)'},
                       {
                           '(ontable b)', '(handempty)', '(clear a)',
                           '(on a b)', '(holding a)', '(clear b)'
                       },
                       {
                           '(handempty)', '(ontable a)', '(holding a)',
                           '(holding b)', '(ontable b)', '(on a a)',
                           '(clear a)', '(on a b)', '(clear b)'
                       }]
    F_sets, A_sets = construct_rpg(task)
    assert len(F_sets) == len(
        expected_F_sets
    ), f"Expected F_sets of length {len(expected_F_sets)}, but got {len(F_sets)}."
    for F_set, expected_F_set in zip(F_sets, expected_F_sets):
        assert F_set == expected_F_set, f"Incorrect F_set: {F_set}"
    expected_A_set_strs = [{'(unstack a b)'},
                           {
                               '(unstack a b)', '(pick-up b)', '(stack a a)',
                               '(stack a b)', '(put-down a)'
                           }]
    assert len(A_sets) == len(
        expected_A_set_strs
    ), f"Expected A_sets of length {len(expected_A_set_strs)}, but got {len(A_sets)}."
    for A_set, expected_A_set_str in zip(A_sets, expected_A_set_strs):
        A_set_str = {a.name for a in A_set}
        assert A_set_str == expected_A_set_str, f"Incorrect A_set: {A_set_str}, expected {expected_A_set_str}"
    return F_sets, [{a.name for a in A_set} for a in A_sets]

construct_rpg_test1()


def construct_rpg_test2():
    task = get_task_definition_str(BW_BLOCKS_DOMAIN, BW_BLOCKS_PROBLEM_2)
    expected_F_sets = [
        {
            '(handempty)', '(clear c)', '(ontable a)', '(on c a)', '(clear b)',
            '(ontable b)'
        },
        {
            '(clear c)', '(holding b)', '(ontable a)', '(holding c)',
            '(clear b)', '(clear a)', '(ontable b)', '(handempty)', '(on c a)'
        },
        {
            '(clear c)', '(holding b)', '(ontable a)', '(holding c)',
            '(on b a)', '(on c c)', '(clear b)', '(holding a)', '(clear a)',
            '(ontable b)', '(handempty)', '(on b b)', '(on b c)', '(on c b)',
            '(on c a)', '(ontable c)'
        },
        {
            '(holding b)', '(ontable a)', '(on c c)', '(clear a)',
            '(ontable b)', '(on b b)', '(on c a)', '(on a b)', '(clear c)',
            '(holding c)', '(on b a)', '(on a a)', '(clear b)', '(holding a)',
            '(on a c)', '(handempty)', '(on b c)', '(on c b)', '(ontable c)'
        },
    ]
    F_sets, A_sets = construct_rpg(task)
    assert len(F_sets) == len(
        expected_F_sets
    ), f"Expected F_sets of length {len(expected_F_sets)}, but got {len(F_sets)}."
    for F_set, expected_F_set in zip(F_sets, expected_F_sets):
        assert F_set == expected_F_set, f"Incorrect F_set: {F_set}"
    expected_A_set_strs = [
        {'(unstack c a)', '(pick-up b)'},
        {
            '(unstack c a)', '(pick-up b)', '(stack b a)', '(stack b c)',
            '(put-down c)', '(stack c a)', '(stack c c)', '(put-down b)',
            '(stack c b)', '(pick-up a)', '(stack b b)'
        },
        {
            '(stack b a)', '(stack b c)', '(unstack b b)', '(stack c c)',
            '(stack c b)', '(unstack c b)', '(unstack b c)', '(stack a b)',
            '(put-down a)', '(stack a c)', '(pick-up a)', '(stack a a)',
            '(unstack c a)', '(pick-up b)', '(put-down c)', '(stack c a)',
            '(put-down b)', '(unstack b a)', '(stack b b)', '(pick-up c)',
            '(unstack c c)'
        },
    ]
    assert len(A_sets) == len(
        expected_A_set_strs
    ), f"Expected A_sets of length {len(expected_A_set_strs)}, but got {len(A_sets)}."
    for A_set, expected_A_set_str in zip(A_sets, expected_A_set_strs):
        assert {a.name for a in A_set
               } == expected_A_set_str, f"Incorrect A_set: {A_set}"
    return F_sets, [{a.name for a in A_set} for a in A_sets]

construct_rpg_test2()

print('Tests passed.')

## Implement h_add


### Question
Given a [Pyperplan Task](https://github.com/aibasel/pyperplan/blob/master/pyperplan/task.py) instance and 
    a state (set of facts or None), return h_add(state).

For reference, our solution is **7** line(s) of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `construct_rpg`. You may not need to use all of them.

In [ ]:
def h_add(task, state=None):
    """Computes h_add for a given Pyperplan Task and state.  If state is None,
    use the initial_state of the task.

    Args:
      task: a pyperplan Task instance
      state: a set of facts (can be None)

    Returns:
      h: an integer
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def h_add_test1():
    return h_add_test(BW_BLOCKS_PROBLEM_1, 3)

h_add_test1()


def h_add_test2():
    return h_add_test(BW_BLOCKS_PROBLEM_2, 5)

h_add_test2()


def h_add_test3():
    return h_add_test(BW_BLOCKS_PROBLEM_3, 21)

h_add_test3()


def h_add_test4():
    return h_add_test(BW_BLOCKS_PROBLEM_4, 30)

h_add_test4()

print('Tests passed.')

## Implement h_ff


### Question
Given a [Pyperplan Task](https://github.com/aibasel/pyperplan/blob/master/pyperplan/task.py) and a state (set of facts or None), return h_ff(state).

For reference, our solution is **26** line(s) of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `construct_rpg`. You may not need to use all of them.

In [ ]:
def h_ff(task, state=None):
    """Computes h_ff for a given task and state.  If state is None, use the
    initial_state of the task.

    Args:
      task: a Pyperplan Task instance
      state: a set of facts (can be None)

    Returns:
      h: an integer
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def h_ff_test1():
    return h_ff_test(BW_BLOCKS_PROBLEM_1, 2)

h_ff_test1()


def h_ff_test2():
    return h_ff_test(BW_BLOCKS_PROBLEM_2, 5)

h_ff_test2()


def h_ff_test3():
    return h_ff_test(BW_BLOCKS_PROBLEM_3, 12)

h_ff_test3()


def h_ff_test4():
    return h_ff_test(BW_BLOCKS_PROBLEM_4, 16)

h_ff_test4()

print('Tests passed.')

## Extend Path in a Robot Configuration Space


### Utilities

### RRT-Related Utilties
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:

from typing import Callable, Sequence, Tuple, List, Optional, Union

import collections


# Useful geometric data structures and methods.
def euclidean_distance(a, b):
    return math.sqrt((a.x - b.x)**2 + (a.y - b.y)**2)


class Obstacle:

    @abstractmethod
    def collide(self, config):
        ...


@dataclasses.dataclass
class Circle(Obstacle):

    x: float
    y: float
    r: float

    def collide(self, config):
        return euclidean_distance(config, self) < self.r


RobotConfig = collections.namedtuple('RobotConfig', ['x', 'y'])

ExtentType = Tuple[Tuple[float, float], Tuple[float, float]]


@dataclasses.dataclass
class RobotPlanningProblem:
    """A robot planning problem definition contains the configuration space of
    the robot and all obstacles represented as geometries in the configuration
    space.

    In this problem set, we will consider a simple 2D map, where the robot configuration
    space is represented as a tuple (x, y).
    """

    initial: RobotConfig = RobotConfig(-8, -8)
    goal: RobotConfig = RobotConfig(8, 8)
    extent: ExtentType = ((-10, -10), (10, 10))
    obstacles: Sequence[Obstacle] = (Circle(0, 0, 4), Circle(-5, -5, 2),
                                     Circle(3, -4, 3), Circle(-3, 4, 1))

    def sample(self) -> RobotConfig:
        """Generate a random sample in the robot configuration space."""
        return RobotConfig(random.uniform(self.extent[0][0], self.extent[1][0]),
                           random.uniform(self.extent[0][1], self.extent[1][1]))

    def distance(self, a: RobotConfig, b: RobotConfig) -> float:
        """Return the distance between two robot configurations.

        Args:
          a, b: two robot configurations.

        Return:
          distance: the distance between two configurations.
        """
        return math.sqrt((a.x - b.x)**2 + (a.y - b.y)**2)

    def collide(self, x: RobotConfig) -> bool:
        """Test if the input robot configuration collides with some obstacles.

        Args:
          x: the input configuration.

        Returns:
          collision: True if a collision happens between the input configuration and one of the obstacle in the environment.
        """
        return any(obs.collide(x) for obs in self.obstacles)


@dataclasses.dataclass(frozen=False, eq=False)
class RRTNode:
    """Node of a rapidly-exploring random tree.

    Each node is associated with a robot configuration.
    It also keeps track of
      - the parent in the tree.
      - a list of children nodes in the tree.

    If the node itself is the node for an RRT, node.parent will be set to None.
    If the node is a leaf node in an RRT, node.children will be an empty list.
    """

    config: RobotConfig
    children: List['RRTNode'] = dataclasses.field(default_factory=list)
    parent: 'RRTNode' = None

    def add_child(self, other: 'RRTNode') -> 'RRTNode':
        """Register another node as the child of the current node.

        Args:
          other: the other RRTNode.

        Returns:
          self
        """
        self.children.append(other)
        return self

    def attach_to(self, parent: 'RRTNode') -> 'RRTNode':
        """Attach the current node to another node. This function will:

          - set the parent of self to parent.
          - add self to parent.children.

        Args:
          parent: the parent node.

        Returns:
          self
        """
        self.parent = parent
        self.parent.add_child(self)
        return self

    def backtrace(self, config: bool = True) -> List[RobotConfig]:
        """Return the path from root to the current node. 

        For example, if the tree is:

            s0 --> s1 -> s2 -> S3 -> s4
            \
              -> s5 -> s6

        Calling s6.backtrace() will return the list: `[s0, s5, s6]`.

        Args:
            config: if True, return a list of robot configurations.
                Otherwise, return the RRTNode list.

        Returns:
            path: the path from root to the current node.
        """
        path = []

        def dfs(x):
            if x.parent is not None:
                dfs(x.parent)
            path.append(x.config if config else x)

        dfs(self)
        return path

    def __repr__(self) -> str:
        return f"RRTNode(config={self.config}, parent={self.parent})"


class RRT:
    """The RRT tree.

    Note that the RRT class does not explicitly manages all nodes in the tree.
    It only keeps track of the root nodes of the tree.
    To get all nodes in an RRT tree, we need to do a tree travesal from the roots.
    We have provided a helper function traverse_rrt_bfs for you to do that.

    There are two useful helper functions in RRT class.
      - extend: which attaches a new node to an existing node in the tree.
      - nearest: which finds the node in the RRT that is most close to the query node.
    See the function docstrings for details.
    """

    def __init__(self, problem: RobotPlanningProblem,
                 roots: Union[RRTNode, Sequence[RRTNode]]):
        """The constructor of RRT takes two parameters:

        Args:
            problem: an instance of the robot planning problem.
                It will define the distance metric in the configuration space.
            roots: a single RRTNode or a list of RRTNode instances as the root(s) of the RRT.
        """
        if isinstance(roots, RRTNode):
            roots = [roots]
        self.roots = list(roots)
        self.problem = problem
        self.size = len(self.roots)

    def extend(self, parent: RRTNode, child_config: RobotConfig) -> RobotConfig:
        """Create a new RRTNode from a robot configuration and set its parent.

        Basically, it creates a new RRTNode instance from child_config and attach this
        new node to the `parent` node. We recommend you to use this function
        instead of doing it yourself because it will help you keep track of the size
        of the RRT, which may be useful for your debugging purpose.

        Args:
          parent: the parent node (in the current RRT).
          child_config: the robot configuration of the new node.

        Returns:
          child: the new node.
        """
        child = RRTNode(child_config).attach_to(parent)
        self.size += 1
        return child

    def nearest(self, config: RobotConfig) -> RRTNode:
        """Return the RRTNode in the tree that is closest to the target
        configuration. We strongly suggest you to take a look at the function
        definition to see the example usage of function `traverse_rrt_bfs` and
        `problem.distance`.

        Args:
          config: the query robot config.

        Returns:
          best_node: a node in the RRT that is closest to the query configuration
            based on the distance metric in the configuration space.
        """
        best_node, best_value = None, np.inf
        for node in traverse_rrt_bfs(self.roots):
            distance = self.problem.distance(node.config, config)
            if distance < best_value:
                best_value = distance
                best_node = node

        return best_node


def traverse_rrt_bfs(nodes: Sequence[RRTNode]) -> List[RRTNode]:
    """Run breath-first search to return all RRTNodes that are descendants of
    the input list of RRTNodes.

    Args:
      nodes: a list of RRTNodes as the seed nodes.

    Returns:
      a list of RRTNodes that are descendants of the input lists (including the input list).
    """
    queue = list(nodes)
    results = []

    while queue:
        x = queue.pop(0)
        results.append(x)
        queue.extend(x.children)

    return results


########## Test-Related Tools for RRT ##########


def check_path_smooth(problem, path, max_diff, eps=1e-5):
    """Check if a path is collision-free and satisfies the max_diff constraint."""
    if path is None or len(path) == 0:
        return False
    for config1, config2 in zip(path[:-1], path[1:]):
        if any(
                abs(c1 - c2) > max_diff + eps
                for c1, c2 in zip(config1, config2)):
            return False
    return all(not problem.collide(config) for config in path)


def check_path_smooth_and_successful(problem: RobotPlanningProblem,
                                     path,
                                     max_diff,
                                     eps=1e-5):
    """Check if a path is collision-free, satisfies the max_diff constraint,
    and connects initial and goal configs."""
    return (path is not None and len(path) > 0 and
            np.allclose(path[0], problem.initial, atol=eps) and
            np.allclose(path[-1], problem.goal, atol=eps) and
            check_path_smooth(problem, path, max_diff, eps=eps))

### RRT Visualization-Related Utilties
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:

import functools


def to_canvas_coord(problem: RobotPlanningProblem, canvas_size: Tuple[int, int],
                    x: float, y: float) -> Tuple[int, int]:
    return (int((x - problem.extent[0][0]) /
                (problem.extent[1][0] - problem.extent[0][0]) * canvas_size[0]),
            canvas_size[1] - int(
                (y - problem.extent[0][1]) /
                (problem.extent[1][1] - problem.extent[0][1]) * canvas_size[1]))


def visualize_map(problem: RobotPlanningProblem,
                  canvas_size: Tuple[int] = (512, 512)):
    """Visualize a 2D environment.

    Args:
      problem: the robot planning problem.
      canvas_size: width and height of the canvas.

    Returns:
      a PIL image of the visualization.
    """
    from PIL import Image, ImageDraw

    image = Image.new('RGB', canvas_size, color='white')
    draw = ImageDraw.Draw(image)

    gen_xy = functools.partial(to_canvas_coord, problem, canvas_size)

    for obs in problem.obstacles:
        if isinstance(obs, Circle):
            AABB = (*gen_xy(obs.x - obs.r, obs.y + obs.r),
                    *gen_xy(obs.x + obs.r, obs.y - obs.r))
            draw.ellipse(AABB, fill='gray', outline='gray')

    return image


def visualize_path(problem: RobotPlanningProblem,
                   path: List[RobotConfig],
                   canvas_size: Tuple[int, int] = (512, 512),
                   show_nodes: bool = True):
    """Visualize a path in a 2D environment. The starting point will be marked
    as blue. The end point will be marked as green.

    Args:
      problem: the robot planning problem.
      path: a list of RobotConfig.
      canvas_size: width and height of the canvas.
      show_nodes: if True, the function will visualize all nodes along the path.

    Returns:
      a PIL image of the visualization.
    """
    from PIL import Image, ImageDraw

    image = visualize_map(problem, canvas_size=canvas_size)
    draw = ImageDraw.Draw(image)

    gen_xy = functools.partial(to_canvas_coord, problem, canvas_size)

    for config1, config2 in zip(path[:-1], path[1:]):
        draw.line(
            (*gen_xy(config1.x, config1.y), *gen_xy(config2.x, config2.y)),
            fill='red')

    if show_nodes:
        for config in path[1:-1]:
            draw.rectangle(
                [
                    *gen_xy(config.x - 0.1, config.y - 0.1),
                    *gen_xy(config.x + 0.1, config.y + 0.1)
                ],
                fill='red',
            )

    draw.rectangle(
        [
            *gen_xy(path[0].x - 0.3, path[0].y - 0.3),
            *gen_xy(path[0].x + 0.3, path[0].y + 0.3)
        ],
        fill='blue',
    )
    draw.rectangle(
        [
            *gen_xy(path[-1].x - 0.3, path[-1].y - 0.3),
            *gen_xy(path[-1].x + 0.3, path[-1].y + 0.3)
        ],
        fill='green',
    )

    return image


def visualize_rrts(problem: RobotPlanningProblem,
                   rrts: List[RRT],
                   canvas_size: Tuple[int, int] = (512, 512),
                   show_nodes: bool = True):
    """Visualize RRTs in a 2D environment. The initial config will be marked as
    blue. The goal config will be marked as green.

    Args:
      problem: the robot planning problem.
      rrts: a list of RRTs. Currently the function supports up to two rrts. The first one will show in red,
        while the second one wil show in blue.
      canvas_size: width and height of the canvas.
      show_nodes: if True, the function will visualize all nodes in the tree.

    Returns:
      a PIL image of the visualization.
    """
    from PIL import ImageDraw

    image = visualize_map(problem, canvas_size=canvas_size)
    draw = ImageDraw.Draw(image)

    gen_xy = functools.partial(to_canvas_coord, problem, canvas_size)

    for rrt, color in zip(rrts, ['red', 'blue']):
        for node in traverse_rrt_bfs(rrt.roots):
            if node.parent is not None:
                draw.line((*gen_xy(node.parent.config.x, node.parent.config.y),
                           *gen_xy(node.config.x, node.config.y)),
                          fill=color)

            if show_nodes:
                draw.rectangle(
                    [
                        *gen_xy(node.config.x - 0.1, node.config.y - 0.1),
                        *gen_xy(node.config.x + 0.1, node.config.y + 0.1)
                    ],
                    fill='red',
                )

    draw.rectangle(
        [
            *gen_xy(problem.initial.x - 0.3, problem.initial.y - 0.3),
            *gen_xy(problem.initial.x + 0.3, problem.initial.y + 0.3)
        ],
        fill='blue',
    )
    draw.rectangle(
        [
            *gen_xy(problem.goal.x - 0.3, problem.goal.y - 0.3),
            *gen_xy(problem.goal.x + 0.3, problem.goal.y + 0.3)
        ],
        fill='green',
    )

    return image

### Question
In this section, you will implement an important utility for doing motion planning in a continuous space: extend_path.
The function takes in a RobotPlanningProblem. It makes a linear interpolation between the initial and goal
configurations in the robot's configuration space.

Its output is a path, that is, a list of robot configurations along the line from `start` to `goal`.
When the `start` and `goal` cannot be directly connected with a line because of collision,
your function should return a "prefix" of the path.
In other words, you should try to move towards the goal until you reach some obstacles in the environment.

Take a close look at the docstring of the `RobotPlanningProblem` class before you implement this function.

Note that:
- Although our test cases are for a two-dimensional configuration $[x, y]$.  
    The code for `extend_path` is written so that it can work for any number of dimensions. 
    Furthermore, the implementation of RRT should also work for any dimension without change.
- Our solution does not use `numpy`. However, if you are comfortable with `numpy`, 
    feel free to use it to shorten your implementation. 
    

For reference, our solution is **18** line(s) of code.

In [ ]:
def extend_path(problem: RobotPlanningProblem, start: RobotConfig,
                end: RobotConfig,
                max_diff: float) -> Tuple[bool, List[RobotConfig]]:
    """Construct a linear path in the configuration space from the initial
    configuration to the goal configuration.

    Your implementation should have two steps.
    Step 1, construct a path that is a linear interpolation between `problem.initial` and `problem.goal`.
    At each step, the movement along x and along y should be both smaller than or equal to max_diff.

    Step 2: check for collisions.
    You should iterate over the configurations in the path generated in step 1,
    if a configuration triggers a collision with an obstacle,
    your algorithm should return the longest prefix path that does not have any collision.
    We have implemented this test for you. But make sure you read the code and understand how it works.

    Note that, even though we have separated linear interpolation and collision checking as two
    separate steps, you can also choose to do them simultaneously in a single for loop.

    Args:
      problem: a RobotPlanningProblem instance, used for collision checking. 
      start: a starting RobotConfig. It should be the first element of your return list. 
      end: an ending RobotConfig. It should be the last element of your return list (if succeed).
      max_diff: the maximum movement in x or y.

    Returns:
      successful: a bool value indicating whether we have found a path from the initial configuration to the goal configuration.
        If a collision happens, you should return False. See the starter code.
      path: a list of RobotConfig: extending from the initial to the goal configuration.
    """
    path = [start]
    # Step 1: Generate a list of robot configurations that connects `start` and `goal`.
    # Your code only needs to take care of max_diff. You can ignore collision checking for now.
    # Your code here...
    assert path[-1] == goal

    # Step 2: Collision checking.
    for i, config in enumerate(path):
        if problem.collide(config):
            return False, path[:i]
    return True, path

### Tests

In [ ]:
def extend_path_test1():
    problem = RobotPlanningProblem()
    success, path = extend_path(problem, problem.initial, problem.goal, 0.1)
    assert not success  # you can't reach the goal with a single linear motion.
    assert len(path) > 10  # you should make some progress.
    assert check_path_smooth(problem, path, 0.1)

extend_path_test1()


def extend_path_test2():
    problem = RobotPlanningProblem(RobotConfig(-8, -8), RobotConfig(-7, 6))
    success, path = extend_path(problem, problem.initial, problem.goal, 0.1)
    assert success
    assert check_path_smooth_and_successful(problem, path, 0.1)

extend_path_test2()

print('Tests passed.')

## Extend Path Visualization


### Utilities

### RRT Visualization-Related Utilties
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:

import functools


def to_canvas_coord(problem: RobotPlanningProblem, canvas_size: Tuple[int, int],
                    x: float, y: float) -> Tuple[int, int]:
    return (int((x - problem.extent[0][0]) /
                (problem.extent[1][0] - problem.extent[0][0]) * canvas_size[0]),
            canvas_size[1] - int(
                (y - problem.extent[0][1]) /
                (problem.extent[1][1] - problem.extent[0][1]) * canvas_size[1]))


def visualize_map(problem: RobotPlanningProblem,
                  canvas_size: Tuple[int] = (512, 512)):
    """Visualize a 2D environment.

    Args:
      problem: the robot planning problem.
      canvas_size: width and height of the canvas.

    Returns:
      a PIL image of the visualization.
    """
    from PIL import Image, ImageDraw

    image = Image.new('RGB', canvas_size, color='white')
    draw = ImageDraw.Draw(image)

    gen_xy = functools.partial(to_canvas_coord, problem, canvas_size)

    for obs in problem.obstacles:
        if isinstance(obs, Circle):
            AABB = (*gen_xy(obs.x - obs.r, obs.y + obs.r),
                    *gen_xy(obs.x + obs.r, obs.y - obs.r))
            draw.ellipse(AABB, fill='gray', outline='gray')

    return image


def visualize_path(problem: RobotPlanningProblem,
                   path: List[RobotConfig],
                   canvas_size: Tuple[int, int] = (512, 512),
                   show_nodes: bool = True):
    """Visualize a path in a 2D environment. The starting point will be marked
    as blue. The end point will be marked as green.

    Args:
      problem: the robot planning problem.
      path: a list of RobotConfig.
      canvas_size: width and height of the canvas.
      show_nodes: if True, the function will visualize all nodes along the path.

    Returns:
      a PIL image of the visualization.
    """
    from PIL import Image, ImageDraw

    image = visualize_map(problem, canvas_size=canvas_size)
    draw = ImageDraw.Draw(image)

    gen_xy = functools.partial(to_canvas_coord, problem, canvas_size)

    for config1, config2 in zip(path[:-1], path[1:]):
        draw.line(
            (*gen_xy(config1.x, config1.y), *gen_xy(config2.x, config2.y)),
            fill='red')

    if show_nodes:
        for config in path[1:-1]:
            draw.rectangle(
                [
                    *gen_xy(config.x - 0.1, config.y - 0.1),
                    *gen_xy(config.x + 0.1, config.y + 0.1)
                ],
                fill='red',
            )

    draw.rectangle(
        [
            *gen_xy(path[0].x - 0.3, path[0].y - 0.3),
            *gen_xy(path[0].x + 0.3, path[0].y + 0.3)
        ],
        fill='blue',
    )
    draw.rectangle(
        [
            *gen_xy(path[-1].x - 0.3, path[-1].y - 0.3),
            *gen_xy(path[-1].x + 0.3, path[-1].y + 0.3)
        ],
        fill='green',
    )

    return image


def visualize_rrts(problem: RobotPlanningProblem,
                   rrts: List[RRT],
                   canvas_size: Tuple[int, int] = (512, 512),
                   show_nodes: bool = True):
    """Visualize RRTs in a 2D environment. The initial config will be marked as
    blue. The goal config will be marked as green.

    Args:
      problem: the robot planning problem.
      rrts: a list of RRTs. Currently the function supports up to two rrts. The first one will show in red,
        while the second one wil show in blue.
      canvas_size: width and height of the canvas.
      show_nodes: if True, the function will visualize all nodes in the tree.

    Returns:
      a PIL image of the visualization.
    """
    from PIL import ImageDraw

    image = visualize_map(problem, canvas_size=canvas_size)
    draw = ImageDraw.Draw(image)

    gen_xy = functools.partial(to_canvas_coord, problem, canvas_size)

    for rrt, color in zip(rrts, ['red', 'blue']):
        for node in traverse_rrt_bfs(rrt.roots):
            if node.parent is not None:
                draw.line((*gen_xy(node.parent.config.x, node.parent.config.y),
                           *gen_xy(node.config.x, node.config.y)),
                          fill=color)

            if show_nodes:
                draw.rectangle(
                    [
                        *gen_xy(node.config.x - 0.1, node.config.y - 0.1),
                        *gen_xy(node.config.x + 0.1, node.config.y + 0.1)
                    ],
                    fill='red',
                )

    draw.rectangle(
        [
            *gen_xy(problem.initial.x - 0.3, problem.initial.y - 0.3),
            *gen_xy(problem.initial.x + 0.3, problem.initial.y + 0.3)
        ],
        fill='blue',
    )
    draw.rectangle(
        [
            *gen_xy(problem.goal.x - 0.3, problem.goal.y - 0.3),
            *gen_xy(problem.goal.x + 0.3, problem.goal.y + 0.3)
        ],
        fill='green',
    )

    return image

### Question
We have created a visualization tool for you in colab to visualize the path. 
You can run the following code blocks and they will generate the visualized the map as well as the path.

In [ ]:
problem = RobotPlanningProblem(RobotConfig(-8, -8), RobotConfig(-7, 6))
# we have set max_diff to 0.5 for better visualization.
success, path = extend_path(problem, problem.initial, problem.goal, 0.5)
visualize_path(problem, path)

## RRT


### Question

In this section, you will implement the RRT algorithm we talked about in class to find a path from one configuration to the goal configuration in a 2D environment.

We have implemented some utility classes (RRTNode, RRT) for you, as well as their corresponding visualization tools.
You should take a close look at their docstrings before you implement this function. 
**To run in Colab, you will need an implementation of `extend_path`, if you did not write your own, use the solution in Catsoop.**

**Hint**: recall that there is a hyperparameter in RRT, which balances how frequently we want to test if the goal configuration can be reached from a node in the current
RRT tree. Our suggested value for this hyperparameter is $0.1$.


For reference, our solution is **19** line(s) of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `extend_path`. You may not need to use all of them.

In [ ]:
def rrt(problem: RobotPlanningProblem,
        max_diff: float,
        nr_iterations: int = 100,
        goal_sample_prob: float = 0.1) -> Tuple[List[RobotConfig], RRT]:
    """Implement RRT algorithm to compute a path from the initial configuration
    to the target configuration. Your implementation should returns a list of
    robot configurations, starting from `problem.initial` and terminating at `proble.goal`.

    Similar to the definition in extend_path, at each step, the movement along x and along y should be both smaller than or equal to max_diff.
    You should use the extend_path utility that you just implemented to check whether there is a path from one node to another.

    Args:
      problem: an RobotPlanningProblem instance, used for configuration sampling and collision checking.
        `problem.initial` should be the first element of your return list.
        `problem.goal` should be the last element of your return list.
      max_diff: the maximum movement in x or y.
      nr_iterations: the number of iterations in RRT.
      goal_sample_prob: at each iteration, with prob p, you should try to construct a path from a node to the goal configuration.
        Otherwise, you should sample a new configuration and try to constrcut a path from a node to it.

    Returns:
      path: a list of RobotConfig as the trajectory. Return None if we can't find such a path after the specified number of iterations.
      rrt: an RRT object. The root should be the intial_config. There must also exist another node for goal_config.
    """
    raise NotImplementedError("Implement me!")

### Tests

In [ ]:
def rrt_test():
    problem = RobotPlanningProblem(RobotConfig(-8, -8), RobotConfig(8, 8))
    path, tree = rrt(problem, 0.5, nr_iterations=200)
    assert check_path_smooth_and_successful(problem, path, 0.5)

rrt_test()

print('Tests passed.')

## RRT Visualization


### Question

We have created a visualization tool for you in colab to visualize the RRT tree during your search. You can run the following code blocks
and they will generate the visualized the map as well as some lines showing the edges in the RRT.

If you run this cell multiple times, you will see different RRT trees due to different random seeds.
You will find that sometimes the tree has very few nodes -- indicating that your RRT algorithm finds a solution very quickly.
Sometimes it takes many iterations to find the solution. This suggests that the algorithm is very sensitive to random seeds,
which is bad for practical usage. Thus, in practice, we shuold always use improved RRT algorithm variants such as BiRRT (bidirectional RRT).

We recommend you try in your colab notebook to see how the hyperparameter `goal_sample_probability` will affect the search process.
Specifically, try to set that probability to 0.05 or 0.2 and visualize your result.


In [ ]:
problem = RobotPlanningProblem()
# we have set max_diff to 0.5 for better visualization.
path, tree = rrt(problem, 0.5)
visualize_rrts(problem, [tree])